[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/enolan/jax-torch-bench/blob/master/JAX_vs_PyTorch_transformer_LM_benchmark.ipynb)

In [1]:
# Config is tuned for a P100, a 16GB GPU. You'll have to reduce the batch sizes to get
# the models running if Colab gives you something smaller.
!nvidia-smi

Sun May 21 13:53:07 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.85.12    Driver Version: 525.85.12    CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   71C    P8    11W /  70W |      0MiB / 15360MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

# Common setup

In [2]:
!wget http://mattmahoney.net/dc/enwik9.zip
!unzip enwik9.zip

--2023-05-21 13:53:07--  http://mattmahoney.net/dc/enwik9.zip
Resolving mattmahoney.net (mattmahoney.net)... 34.198.1.81
Connecting to mattmahoney.net (mattmahoney.net)|34.198.1.81|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 322592222 (308M) [application/zip]
Saving to: ‘enwik9.zip.2’

enwik9.zip.2        100%[===================>] 307.65M  31.8MB/s    in 10s     

2023-05-21 13:53:18 (29.8 MB/s) - ‘enwik9.zip.2’ saved [322592222/322592222]

Archive:  enwik9.zip
replace enwik9? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
  inflating: enwik9                  y



In [3]:
!pip install optax flax -q
!pip install -U numpy -q

In [4]:
!git clone https://github.com/enolan/jax-torch-bench.git

fatal: destination path 'jax-torch-bench' already exists and is not an empty directory.


In [5]:
%cd jax-torch-bench

/content/jax-torch-bench


In [6]:
from config import ModelConfig

enwik9 = "/content/enwik9"

# This is the highest batch size PyTorch can handle, the JAX model can do 79
cfg = ModelConfig(seq_len=256, n_layers=12, d_model=512, num_heads=8, ff_dim=3072, dropout=0.1, batch_size=63, learning_rate=1e-3)

# JAX model

In [8]:
# !pip install jax_model

In [9]:
from jax_model import *

In [10]:
# Set up the model
params, model, optimizer, opt_state, sample = setup_all(cfg)

In [11]:
# This will run the training loop indefinitely. Hit the stop button to abort.
params, opt_state = train_loop(model, optimizer, opt_state, params, cfg, enwik9)

In [ ]:
sample(params, "'''Star Trek'''")

# PyTorch model

Since JAX preallocates all GPU memory, you'll need to restart the runtime (Runtime -> Restart runtime) to try the PyTorch model. Then rerun the config setup cell before running the ones below.

In [7]:
from pytorch_model import *

In [8]:
lm = LM(cfg).cuda()

In [ ]:
train_loop(lm, cfg, enwik9)

In [ ]:
lm.sample("'''Star Trek'''")